In [1]:
from selenium import webdriver
import datetime
from bs4 import BeautifulSoup
import urllib.request
import json

In [2]:
browser = webdriver.Chrome('driver/chromedriver') 

In [8]:
def getPage(flightID,date=datetime.datetime.now().date().strftime("%Y%m%d")):
    diff = datetime.datetime.strptime(date, "%Y%m%d").date()-datetime.datetime.now().date()
    if diff > datetime.timedelta(-14):
        link = 'https://uk.flightaware.com/live/flight/{}/history/{}/'.format(flightID,date)
        print(link)
        browser.get(link)
        return True
    else:
        return False

def getSummary():
    flightSummary = browser.find_element_by_id('flightPageTourStep1')
    flightSummarySoup = BeautifulSoup(flightSummary.get_attribute('innerHTML'), 'html.parser')
    sourceDest = flightSummarySoup.find('div',{"class": "flightPageSummaryAirports"})
    sourceIATA = sourceDest.find('div',{'class':'flightPageSummaryOrigin'}).find('span',{'class':'displayFlexElementContainer'}).text
    sourceFName = sourceDest.find('div',{'class':'flightPageSummaryOrigin'}).find('span',{'class':'flightPageSummaryCity'}).text
    destIATA = sourceDest.find('div',{'class':'flightPageSummaryDestination'}).find('span',{'class':'displayFlexElementContainer'}).text
    destFName = sourceDest.find('div',{'class':'flightPageSummaryDestination'}).find('span',{'class':'flightPageSummaryCity'}).text
    dateOfDeparture = flightSummarySoup.find('span',{'class':'flightPageSummaryDepartureDay'}).text.split(' ')[-1]
    dayOfDeparture =  flightSummarySoup.find('span',{'class':'flightPageSummaryDepartureDay'}).text.split(' ')[0]
    timeOfDeparture =  flightSummarySoup.find('span',{'class':'flightPageSummaryDeparture'}).text.split('\n\n\n')[0]
    dateOfArrival = flightSummarySoup.find('span',{'class':'flightPageSummaryArrivalDay'}).text.split(' ')[-1]
    dayOfArrival =  flightSummarySoup.find('span',{'class':'flightPageSummaryArrivalDay'}).text.split(' ')[0]
    timeOfArrival =  flightSummarySoup.find('span',{'class':'flightPageSummaryArrival'}).text.split('\n\n\n')[0]
    departureDelay = flightSummarySoup.find('span',{'class':'flightPageDepartureDelayStatus'}).text.strip("()")
    arrivalDelay = flightSummarySoup.find('span',{'class':'flightPageArrivalDelayStatus'}).text.strip("()")
    totalFlightTime = flightSummarySoup.find('span',{'class':'flightPageProgressTotal'}).text.strip().replace('\n total flight time','')
    AboutFlight  = browser.find_element_by_class_name('flightPageSummaryContainer')
    AboutFlightSoup = BeautifulSoup(AboutFlight.get_attribute('innerHTML'), 'html.parser')
    currentStatus = AboutFlightSoup.find('div',{'class':'flightPageSummaryStatus'})
    summary = {}
    summary['totalTime'] = totalFlightTime
    summary['status'] = currentStatus.contents[0]
    if len(currentStatus.contents)>1:
        summary['sub-status'] = currentStatus.contents[1].text
    source = {}
    source['name'] = sourceFName.strip()
    source['airportCode'] = sourceIATA.strip()
    summary['source'] = source
    destination = {}
    destination['name'] = destFName.strip()
    destination['airportCode'] = destIATA.strip()
    summary['destination'] = destination
    departure = {}
    departure['day'] = dayOfDeparture.strip()
    departure['date'] = dateOfDeparture.strip()
    departure['time'] = timeOfDeparture.strip()
    departure['status'] = departureDelay.title()
    summary['departure'] = departure
    arrival = {}
    arrival['day'] = dayOfArrival.strip()
    arrival['date'] = dateOfArrival.strip()
    arrival['time'] = timeOfArrival.strip()
    arrival['status'] = arrivalDelay.title()
    summary['arrival'] = arrival
#     print(json.dumps(summary,indent=4))
    return summary

In [9]:
getPage('UAE353')
getSummary()

https://uk.flightaware.com/live/flight/UAE353/history/20180522/


{'totalTime': '6h 58m',
 'status': 'Arrived 13 minutes ago',
 'sub-status': 'Terminal 3',
 'source': {'name': 'Singapore', 'airportCode': 'SIN'},
 'destination': {'name': 'Dubai, United Arab Emirates', 'airportCode': 'DXB'},
 'departure': {'day': 'Tuesday',
  'date': '22-May-2018',
  'time': '10:35 +08',
  'status': 'On Time'},
 'arrival': {'day': 'Tuesday',
  'date': '22-May-2018',
  'time': '13:33 +04',
  'status': '7 Minutes Early'}}

In [10]:
getPage('IGO215')
getSummary()

https://uk.flightaware.com/live/flight/IGO215/history/20180522/


{'totalTime': '1h 50m',
 'status': 'En route and on time',
 'sub-status': 'Landing in 45 minutes',
 'source': {'name': 'Kochi / Nedumbassery, India', 'airportCode': 'COK'},
 'destination': {'name': 'Mumbai, India', 'airportCode': 'BOM'},
 'departure': {'day': 'Tuesday',
  'date': '22-May-2018',
  'time': '14:10 IST',
  'status': 'On Time'},
 'arrival': {'day': 'Tuesday',
  'date': '22-May-2018',
  'time': '16:00 IST',
  'status': 'On Time'}}